# A single particle in a double well potential

This example will run [Langevin dynamics](https://en.wikipedia.org/wiki/Langevin_dynamics) for a single particle in a double well potential. The double well potential ($V_\text{pot}$) is given by

$$V_\text{pot} = a x^4 - b (x - c)^2,$$

where $a$, $b$, and $c$ are potential parameters, and $x$ is the position of the particle.

## Importing turtlemd and displaying the potential function

In [ ]:
import black
import jupyter_black

jupyter_black.load(
    lab=False,
    line_length=79,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

In [ ]:
# Imports from turtlemd:
from turtlemd.system import Box, System, Particles
from turtlemd.potentials.well import DoubleWell
from turtlemd.integrators import LangevinIntertia
from turtlemd.simulation import MDSimulation

In [ ]:
# Imports for plotting and numerics
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns

%matplotlib notebook
sns.set_context("notebook")

To display the potential energy as a function of the position, we first initialize the potential:

In [ ]:
potential = DoubleWell(a=1.0, b=2.0, c=0.0)
print(potential)

Next, we create a method we can reuse for calculating the potential energy.
Since the potential energy functions assume they will operate on a system, we also have to create a system with a box and one particle. After that, we move
the particle around to calculate the potential energy at different locations.

In [ ]:
def calculate_potential_curve(potential):
    """Calculate the potential energy for a 1D potential"""
    box = Box(periodic=[False])
    particles = Particles(dim=box.dim)
    particles.add_particle(pos=0.0)
    system = System(box=box, particles=particles, potentials=[potential])
    pos = np.linspace(-1.7, 1.7, 100)
    vpot = []
    for xi in pos:
        system.particles.pos[0] = xi
        vpot.append(system.potential())
    return pos, vpot


fig, ax = plt.subplots(constrained_layout=True)
pos, vpot = calculate_potential_curve(potential)
ax.plot(pos, vpot)
ax.set(xlabel="Position (arbitrary unit)", ylabel="Energy (arbitrary unit)")
sns.despine(fig=fig)

## Running Langevin dynamics for one particle

To run the dynamics for a single particle, we will create a system (with a box and a particle), a Langevin integrator and a MD simulation:

In [ ]:
# Set up the system:
box = Box(periodic=[False])
particles = Particles(dim=box.dim)
particles.add_particle(pos=-1.0)
system = System(box=box, particles=particles, potentials=[potential])

# Set up the integrator:
integrator = LangevinIntertia(
    timestep=0.002,
    gamma=0.3,
    beta=1.0 / 0.07,
    seed=0,
)

# Create a simulation:
simulation = MDSimulation(system=system, integrator=integrator, steps=1000)

To run the simulation you can use ``simulation.run()``. Here, we will use that and animate the location of the particle:

In [ ]:
# First, set up for plotting
fig, ax = plt.subplots(constrained_layout=True)

(line,) = ax.plot(pos, vpot)  # A line for the potential
point = ax.scatter(
    [], [], s=100, marker="o"
)  # A point for showing the particle
ax.set_xlim(-1.8, 1.8)
ax.set_ylim(-1.1, 2.6)
ax.set(xlabel="Position (arbitrary unit)", ylabel="Energy (arbitrary unit)")
sns.despine(fig=fig)


simulation = MDSimulation(system=system, integrator=integrator, steps=100000)


def init():
    return [point]


def update(frame, system, simulation, point):
    for _ in range(10):
        simulation.step()

    pos = system.particles.pos.flatten()[0]
    vpot = system.particles.v_pot
    point.set_offsets([pos, vpot])
    return [point]


anim = FuncAnimation(
    fig,
    update,
    frames=int(simulation.steps / 10) + 1,
    fargs=[system, simulation, point],
    repeat=False,
    interval=1,
    blit=True,
    init_func=init,
)

plt.show()